# 사용자 데이타에 기반한 RAG(Retrieval-Augmented Generation) 를 사용하여 Question Answering
- 원본 코드
    - https://github.com/aws/amazon-sagemaker-examples/blob/main/introduction_to_amazon_algorithms/jumpstart-foundation-models/question_answering_retrieval_augmented_generation/question_answering_langchain_jumpstart.ipynb

# 1. 기본 환경 설정

In [2]:
%load_ext autoreload
%autoreload 2

# src 폴더 경로 설정
import sys
sys.path.append('../common_code')

In [3]:
import time
import sagemaker, boto3, json
from sagemaker.session import Session
from sagemaker.model import Model
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base


sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()
model_version = "*"

In [4]:
%store -r embedding_model_endpoint_name

print("embedding_model_endpoint_name: \n", embedding_model_endpoint_name)


embedding_model_endpoint_name: 
 KoSimCSE-roberta-2023-05-31-08-36-23


## 모델 정보 입력
- SageMaker 엔드포인트 ARN 입력 등

In [5]:
_MODEL_CONFIG_ = {
    "KoAlpaca-12-8B": {
        "instance type": "ml.g5.12xlarge",
        "endpoint_name" : "KoAlpaca-12-8B-2023-05-30-15-03-24",
        "env": {"TS_DEFAULT_WORKERS_PER_MODEL": "1"},
        "parse_function": "parse_response_model_KoAlpaca",
        "prompt": """Answer based on context:\n\n{context}\n\n{question}""",
    },
    "KoSimCSE-roberta": {
        "instance type": "ml.g5.12xlarge",
        "endpoint_name" : "KoSimCSE-roberta-2023-05-31-08-36-23",        
        "env": {"TS_DEFAULT_WORKERS_PER_MODEL": "1"},
    },
}

# 2. LLM 에 Context 없이 추론 테스트

In [6]:
question = "아마존 스토어에서 무엇을 판매할 수 있나요?"
# question = "아마존 매장에서 상품을 판매하려면 어떻게 해야 하나요?"
c = None
# prompt_wo_c = f"### 질문: {q}\n\n### 맥락: {c}\n\n### 답변:" if c else f"### 질문: {q}\n\n### 답변:" 
prompt_wo_c = f"### question: {question}\n\n### context: {c}\n\n### answer:" if c else f"### question: {question}\n\n### answer:" 
print("prompt_wo_c: \n", prompt_wo_c)

prompt_wo_c: 
 ### question: 아마존 스토어에서 무엇을 판매할 수 있나요?

### answer:


In [7]:

from inference_lib import invoke_inference, query_endpoint_with_text_payload
from inference_lib import parse_response_text_model

model_id = "KoAlpaca-12-8B"
endpoint_name = _MODEL_CONFIG_[model_id]["endpoint_name"]

query_response = query_endpoint_with_text_payload(
    prompt_wo_c, endpoint_name=endpoint_name, 
)

query_response = parse_response_text_model(query_response)
print(query_response)

### question: 아마존 스토어에서 무엇을 판매할 수 있나요?

### answer: 아마존은 광범위한 상품 라인을 보유하고 있습니다. 서적, 컴퓨터 하드웨어 및 소프트웨어, 캠핑 장비, 음악 및 비디오, 가전제품, 건강 및 피트니스, 완구 및 게임, 가구, 화장품, 의류 및 신발 등 다양한 제품을 판매하고 있습니다.


# 3. 데이터 준비

In [8]:
import glob
import os
import pandas as pd

all_files = glob.glob(os.path.join("../Data/", "amazon_faq_ko.csv"))

df_knowledge = pd.concat(
    (pd.read_csv(f )for f in all_files),
    axis=0,
    ignore_index=True,
)

In [9]:
df_knowledge.drop(["Question"], axis=1, inplace=True)
df_knowledge.rename(columns={"Answer": "Context"}, inplace=True)

In [10]:
file_path = "rag_data/amazon_faq_ko_processed_data.csv"
# df_knowledge.to_csv(file_path, header=False, index=False)
df_knowledge.to_csv(file_path, header=True, index=False)

참고
- Lang Chain CSV Loader Code
    - https://github.com/hwchase17/langchain/blob/master/langchain/document_loaders/csv_loader.py

In [11]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path , encoding="utf-8")
documents = loader.load()
documents[0:3]


[Document(page_content='Context: 아마존에 등록하면 한 개 또는 수천 개의 아이템을 유연하게 판매할 수 있습니다. 필요에 따라 셀링 플랜을 선택하면 언제든지 플랜을 변경할 수 있습니다.셀러 센트럴을 사용하여 상품 리스팅을 생성하십시오.이 퀵 스타트 스타일 가이드에 따라 멋진 상품 상세 페이지를 만드세요. Amazon Ads 또는 기타 마케팅 채널을 통해', metadata={'source': 'rag_data/amazon_faq_ko_processed_data.csv', 'row': 0}),
 Document(page_content='Context: 가능성은 사실상 무한합니다.판매할 수 있는 품목은 상품, 상품 카테고리 및 브랜드에 따라 다릅니다.모든 셀러가 이용할 수 있는 카테고리도 있고, 프로페셔널 셀러 계정이 필요한 카테고리도 있고, 판매 승인이 필요한 카테고리도 있고, 타사 셀러가 판매할 수 없는 상품을 포함하는 카테고리도 있습니다.', metadata={'source': 'rag_data/amazon_faq_ko_processed_data.csv', 'row': 1}),
 Document(page_content='Context: “일부 상품은 법률 또는 규제 제한 (예: 처방약) 또는 아마존 정책 (예: 범죄 현장 사진) 을 준수하는 것으로 리스팅되지 않을 수 있습니다.자세한 내용은 셀러 센트럴 도움말을 방문하여 제한 사항 및 특정 카테고리의 신상품에 대한 승인을 요청하는 방법에 대해 자세히 알아보십시오. Fulfillment by Amazon', metadata={'source': 'rag_data/amazon_faq_ko_processed_data.csv', 'row': 2})]

# 4 SageMaker Endpoint Wrapper 준비

## SageMaker LLM Wrapper

In [12]:
from langchain.llms.sagemaker_endpoint import SagemakerEndpoint

In [13]:
from inference_lib import KoAlpacaContentHandler
_KoAlpacaContentHandler = KoAlpacaContentHandler()

In [14]:
parameters = {}

sm_llm = SagemakerEndpoint(
    endpoint_name=_MODEL_CONFIG_["KoAlpaca-12-8B"]["endpoint_name"],
    region_name=aws_region,
    model_kwargs=parameters,
    content_handler=_KoAlpacaContentHandler,
)

## SageMaker Embedding Model Wrapper

In [15]:
from inference_lib import SagemakerEndpointEmbeddingsJumpStart
from inference_lib import KoSimCSERobertaContentHandler

In [16]:

_KoSimCSERobertaContentHandler = KoSimCSERobertaContentHandler()

# content_handler = ContentHandler()

embeddings = SagemakerEndpointEmbeddingsJumpStart(
    endpoint_name=_MODEL_CONFIG_["KoSimCSE-roberta"]["endpoint_name"],
    region_name=aws_region,
    content_handler=_KoSimCSERobertaContentHandler,
)

# 5. Vector Store 생성
- FAISS Vector Store 생성

In [17]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import Chroma, AtlasDB, FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain


In [18]:
index_creator = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding=embeddings,
    text_splitter=CharacterTextSplitter(chunk_size=300, chunk_overlap=0),

)

In [19]:
index = index_creator.from_loaders([loader])

/root/aws-ai-ml-workshop-kr/sagemaker/generative-ai/1-Chatbot/2-Lab02-RAG-LLM/../common_code/inference_lib.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ndim = np.array(response_json).ndim


In [20]:
index.vectorstore.index_to_docstore_id

{0: '6cacc41c-7fc1-4efa-babd-60e3c06b5de1',
 1: 'e9501505-f408-4f4a-ab2b-642263435101',
 2: '2eb1c7f7-8e77-47f8-973a-a9b0265371f8',
 3: '1cc03d3b-4089-4d27-84ad-715c866f8a14',
 4: 'a6c1d824-d1ca-4124-bb9b-12c11f20ebe4',
 5: 'b7a5d7aa-7b1f-433d-8342-18fd255c8aeb',
 6: 'c4e87630-171f-4e0a-8f05-c693641a6b29',
 7: '468623bf-ba0b-480f-9f7c-617ddc506249',
 8: 'bc466d8f-ebf7-40db-842b-e674b3898389',
 9: '1ea23a0a-0f00-4021-adf7-ac8daf3d97b6',
 10: 'aff6926b-74d8-441d-b35b-50f8922f023f',
 11: '6deabc44-2998-4b35-be69-e204084e95dd'}

# 6. 다른 프롬프트로 QA 애플리케이션 테스트

In [21]:
docsearch = FAISS.from_documents(documents, embeddings)

## 첫번째 질문

In [22]:
question1 = question
print("question1: \n" , question1)

question1: 
 아마존 스토어에서 무엇을 판매할 수 있나요?


Send the top 3 most relevant docuemnts and question into LLM to get a answer.

In [23]:
docs = docsearch.similarity_search_with_score(question1, k=3)
docs

[(Document(page_content='Context: 아마존 매장에서 매출을 늘리는 방법에는 여러 가지가 있습니다.Fulfillment by Amazon (아마존 주문처리 서비스) 을 통해 고객에게 프라임 배송을 제공하여 프라임 회원에게 어필하십시오.또한 다음과 같은 작업을 수행할 수 있습니다.\r\n프로모션, 쿠폰 또는 라이트닝 딜을 제공하여 광고 판도를 높이세요.\r\n브랜드', metadata={'source': 'rag_data/amazon_faq_ko_processed_data.csv', 'row': 11}),
  143.54308),
 (Document(page_content='Context: 아마존 스토어에는 신규 판매자에게 많은 기회가 있습니다.판매할 수 있는 품목은 상품, 카테고리 및 브랜드에 따라 다릅니다.모든 셀러가 이용할 수 있는 카테고리도 있고 프로페셔널 셀러 계정이 필요한 카테고리도 있습니다.특정 상품은 판매 승인이 필요하며 다른 카테고리에는 타사 판매자가 판매할 수 없는 상품이 포함됩니다.어떤', metadata={'source': 'rag_data/amazon_faq_ko_processed_data.csv', 'row': 4}),
  153.44637),
 (Document(page_content='Context: 아마존에 등록하면 한 개 또는 수천 개의 아이템을 유연하게 판매할 수 있습니다. 필요에 따라 셀링 플랜을 선택하면 언제든지 플랜을 변경할 수 있습니다.셀러 센트럴을 사용하여 상품 리스팅을 생성하십시오.이 퀵 스타트 스타일 가이드에 따라 멋진 상품 상세 페이지를 만드세요. Amazon Ads 또는 기타 마케팅 채널을 통해', metadata={'source': 'rag_data/amazon_faq_ko_processed_data.csv', 'row': 0}),
  154.03288)]

In [53]:
def make_prompt(doc, question):
    context = docs[2][0].page_content
    # prompt = f'{question} 다음의 Context 를 이용하여 답해주세요. {docs[0].page_content}'
#     prompt = f"""Answer based on Context:\n\n### {context}\n\n### Question: {question}\n\n### Answer:"""
#    prompt = f"""주어진 Context 에 기반하여 Question에 Answer 하세요 :\n\n### {context}\n\n### Question: {question}\n\n### Answer:"""
    prompt = f"""주어진 Context 에 기반하여 질문에 답변 하세요 :\n\n### {context}\n\n### 질문: {question}\n\n### 답변:"""    
    print("######## prompt : ########## \n\n", prompt)
    
    return prompt

prompt = make_prompt(docs[2][0].page_content, question1)

######## prompt : ########## 

 주어진 Context 에 기반하여 질문에 답변 하세요 :

### Context: 아마존 셀러로 시작하려면 먼저 셀링 플랜을 선택하고 아마존 셀링 계정을 설정하십시오.등록을 완료하려면 은행 계좌 번호 및 은행 라우팅 번호, 청구 가능한 신용 카드, 정부 발행 국가 ID, 세금 정보 및 전화번호에 액세스할 수 있는지 확인하십시오.

### 질문: 아마존 스토어에서 무엇을 판매할 수 있나요?

### 답변:


In [54]:
query_response = query_endpoint_with_text_payload(
    prompt, endpoint_name=endpoint_name, 
)

query_response = parse_response_text_model(query_response)
print(query_response)

주어진 Context 에 기반하여 질문에 답변 하세요 :

### Context: 아마존 셀러로 시작하려면 먼저 셀링 플랜을 선택하고 아마존 셀링 계정을 설정하십시오.등록을 완료하려면 은행 계좌 번호 및 은행 라우팅 번호, 청구 가능한 신용 카드, 정부 발행 국가 ID, 세금 정보 및 전화번호에 액세스할 수 있는지 확인하십시오.

### 질문: 아마존 스토어에서 무엇을 판매할 수 있나요?

### 답변: 아마존 스토어에서는 아마존 자체 판매하는 다양한 상품군을 비롯하여, 다른 판매자들이 판매하는 다양한 제품들을 판매할 수 있습니다. 전자제품, 가전제품, 건강 및 미용 제품, 완구, 서적, 스포츠 용품, 주방 용품, 시계 류, 보석, 가구, 카펫, 신발 등 다양한 제품을 판매할 수 있습니다. 


## 두번째 질문

In [55]:
question2 = "아마존 매장에서 상품을 판매하려면 어떻게 해야 하나요?"


In [56]:
docs = docsearch.similarity_search_with_score(question2, k=3)
docs

[(Document(page_content='Context: 아마존 매장에서 매출을 늘리는 방법에는 여러 가지가 있습니다.Fulfillment by Amazon (아마존 주문처리 서비스) 을 통해 고객에게 프라임 배송을 제공하여 프라임 회원에게 어필하십시오.또한 다음과 같은 작업을 수행할 수 있습니다.\r\n프로모션, 쿠폰 또는 라이트닝 딜을 제공하여 광고 판도를 높이세요.\r\n브랜드', metadata={'source': 'rag_data/amazon_faq_ko_processed_data.csv', 'row': 11}),
  121.0862),
 (Document(page_content='Context: 책을 판매하려면:\r\n판매하려는 책 유형 결정\r\n아마존 셀러 계정 생성\r\n월 사용료를 일정하게 지불하는 종량제 일반 셀링 플랜 또는 프로페셔널 셀링 플랜 중에서 선택하세요.\r\n주문 처리 방법 알아보기\r\n도서 가격 설정\r\n책 목록\r\n온라인으로 책을 홍보하세요\r\n책 판매 시 포장 및 배송\r\n온라인으로 책을 판매하는 것은 수익성', metadata={'source': 'rag_data/amazon_faq_ko_processed_data.csv', 'row': 8}),
  143.99666),
 (Document(page_content='Context: 아마존 셀러로 시작하려면 먼저 셀링 플랜을 선택하고 아마존 셀링 계정을 설정하십시오.등록을 완료하려면 은행 계좌 번호 및 은행 라우팅 번호, 청구 가능한 신용 카드, 정부 발행 국가 ID, 세금 정보 및 전화번호에 액세스할 수 있는지 확인하십시오.', metadata={'source': 'rag_data/amazon_faq_ko_processed_data.csv', 'row': 5}),
  150.07126)]

In [57]:
prompt = make_prompt(docs[2][0].page_content, question1)

######## prompt : ########## 

 주어진 Context 에 기반하여 질문에 답변 하세요 :

### Context: 아마존 셀러로 시작하려면 먼저 셀링 플랜을 선택하고 아마존 셀링 계정을 설정하십시오.등록을 완료하려면 은행 계좌 번호 및 은행 라우팅 번호, 청구 가능한 신용 카드, 정부 발행 국가 ID, 세금 정보 및 전화번호에 액세스할 수 있는지 확인하십시오.

### 질문: 아마존 스토어에서 무엇을 판매할 수 있나요?

### 답변:


In [58]:
query_response = query_endpoint_with_text_payload(
    prompt, endpoint_name=endpoint_name, 
)

query_response = parse_response_text_model(query_response)
print(query_response)

주어진 Context 에 기반하여 질문에 답변 하세요 :

### Context: 아마존 셀러로 시작하려면 먼저 셀링 플랜을 선택하고 아마존 셀링 계정을 설정하십시오.등록을 완료하려면 은행 계좌 번호 및 은행 라우팅 번호, 청구 가능한 신용 카드, 정부 발행 국가 ID, 세금 정보 및 전화번호에 액세스할 수 있는지 확인하십시오.

### 질문: 아마존 스토어에서 무엇을 판매할 수 있나요?

### 답변: 아마존 스토어에서는 여러 다양한 제품들을 판매할 수 있습니다. 가전제품, 건강 및 미용 제품, 장난감, 게임, 인테리어 제품 등 다양한 제품들을 판매할 수 있으며, 단일 제품에서 종합적인 제품까지 다양한 제품 구색을 갖출 수 있습니다.


# 트러블 슈팅: LangChain 한글 문제 발생

In [30]:
# prompt_template = """Answer based on context:\n\n{context}\n\n{question}"""

# PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
# PROMPT

In [31]:
# chain = load_qa_chain(llm=sm_llm, prompt=PROMPT)

In [32]:
# result = chain({"input_documents": docs, "question": question}, return_only_outputs=True)[
#     "output_text"
# ]
# result